In [14]:
import csv
import math

import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.image import imread
from skimage import data as skim_data 
from skimage import io, color, exposure
from skimage.transform import resize, rescale, rotate, setup, warp, AffineTransform

import numpy as np

from keras import applications
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Input
from keras.initializers import glorot_uniform
from keras.applications.vgg16 import preprocess_input, VGG16
from keras.applications.inception_resnet_v2 import InceptionResNetV2

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn import datasets, linear_model
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
import csv

## Extract and Save Features ##

We run the dataset through VGG-16, extracting the activations of the last feature layer as our high-dimensional feature vectors. We then reduce the dimensionality using PCA.

In [3]:
from skimage.feature import hog
data_path = "../Data/"

im_prices = np.zeros(7674)
images = []
with open(data_path + "updated_ecommerce_data.csv", "r") as file:
    counter = -1
    for line in file:
        if counter == -1:
            counter += 1
            continue
        line_arr = line.split(',')
        img_path = data_path + 'Images/' + line_arr[1]+ '_0' + '.jpg'
        img_file = imread(img_path)
        curr_im = color.rgb2gray(img_file)
        im_prices[counter] = int(line_arr[4])
        counter+=1
        images.append(curr_im)

FileNotFoundError: [Errno 2] No such file or directory: '../Data/Images/cc51c3dd925d9d84e2fb28c7918dbbc3_0.jpg'

In [4]:
hog_array = np.zeros((len(images), 224*224))
print("hogarray")
print(hog_array)

for i, image in enumerate(images):
    fd, hog_image = hog(image, orientations=8, pixels_per_cell=(32, 32),
                        cells_per_block=(1, 1), visualise=True, block_norm='L2-Hys')
    flattened_len = int(hog_image.shape[0]) * int(hog_image.shape[1])
    pixels = np.reshape(hog_image, (-1, flattened_len))
    hog_array[i, :] = pixels
    
    if i % 1000 == 0:
        print(i)

pca = PCA(n_components=2)
pca.fit(hog_array)
hog_images_compressed = pca.transform(hog_array)
print(hog_images_compressed)

pca = PCA(n_components=200)
pca.fit(hog_array)
hog_images_compressed = pca.transform(hog_array)

np.save("ecommerce_linreg_hog_pca_features", hog_images_compressed)
np.save("ecommerce_linreg_hog_pca_components", pca.components_)

hogarray
[]


ValueError: Found array with 0 sample(s) (shape=(0, 50176)) while a minimum of 1 is required.

In [5]:
ecommerce_indices = np.random.permutation(7674)
ecommerce_train_indices = ecommerce_indices[:6139]
ecommerce_test_indices = ecommerce_indices[6140:]

np.save("ecommerce_train_indices", ecommerce_train_indices)
np.save("ecommerce_test_indices", ecommerce_test_indices)

In [6]:
prices = []
image_paths = []

data_path = "../Data/Images/"
with open("../Data/updated_ecommerce_data.csv") as file:
    reader = csv.reader(file)
    headers = next(reader, None)
    i = -1
    for row in reader:
        i += 1
        index = row[0]
        uniq_id = row[1]
        msrp = int(row[4])
        
        image_path = data_path + uniq_id+ '_0' + '.jpg'
        image_paths.append(image_path)
        prices.append(int(msrp))

train_indices = np.load("ecommerce_train_indices.npy")
test_indices = np.load("ecommerce_test_indices.npy")

# X_train, X_test, y_train, y_test = train_test_split(image_paths, prices, test_size=0.20)


ValueError: invalid literal for int() with base 10: 'Black'


## Initialize Image Generator ##
Due to the size of our dataset (>20,000 images), we cannot read all images into memory. Thus, we write our own image generator, which is a Python generator that reads images a minibatch at a time, preprocessing them and returning the input data and price labels as input to the neural network.

In [7]:
def image_generator(indices, batch_size):

    num_batches = int(len(indices) / batch_size)
    
    while True:
        for batch_i in range(num_batches):
            if batch_i == num_batches - 1:
                # special case: return as many as possible
                start_i = batch_i * batch_size
                batch_indices = indices[start_i:]
                
                X = np.zeros((len(batch_indices), 224, 224, 3))
                Y = np.zeros((len(batch_indices), 1))
            
            else:
                start_i = batch_i * batch_size
                end_i = start_i + batch_size

                batch_indices = indices[start_i:end_i]

                X = np.zeros((batch_size, 224, 224, 3))
                Y = np.zeros((batch_size, 1))
            
            for i, index in enumerate(batch_indices):
                img = image.load_img(image_paths[index], target_size=(224, 224))
                X[i, :, :, :] = image.img_to_array(img)                
                Y[i] = prices[index]
            
            # use vgg16 preprocessing
            X = preprocess_input(X)
            
            yield (X, Y)


## Hyperparameters ##

We tune hyperparameters using grid search and random search, modifying one hyperparameter at a time while keeping the others constant.

In [8]:
# Hyperparameters

num_settings = 1

hp_dropout = [0.2] * num_settings

#RMSprop
hp_lr = [0.01] * num_settings
hp_rho = [0.9] * num_settings
hp_epsilon = [1e-07] * num_settings
hp_decay = [0.0] * num_settings

# Number of hidden units
hp_hidden = [256] * num_settings

# Minibatch size
hp_mbsize = [64] * num_settings

num_epochs = 20

In [9]:
# store the results of each setting
train_losses = np.zeros(num_settings)
dev_losses = np.zeros(num_settings)

for setting in range(num_settings):
    # build the VGG16 network
    input_tensor = Input(shape=(224,224,3))
    model = VGG16(weights='imagenet', include_top=False, input_tensor = input_tensor)
    
    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=(model.output_shape[1:])))


    # Output layer
    # We do random weight intialization
    top_model.add(Dropout(hp_dropout[setting]))
    top_model.add(Dense(hp_hidden[setting], activation='relu', kernel_initializer='glorot_uniform'))
    top_model.add(Dense(1, activation='linear', name='output', kernel_initializer='glorot_uniform'))
    
    # add the model on top of the convolutional base
    new_model = Model(inputs= model.input, outputs = top_model(model.output))

    new_model.layers


2021-09-21 17:29:55.786117: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [11]:
new_model.layers

In [20]:
# store the results of each setting
train_losses = np.zeros(num_settings)
dev_losses = np.zeros(num_settings)

for setting in range(num_settings):
    # build the VGG16 network
    input_tensor = Input(shape=(299,299,3))
    model = InceptionResNetV2(weights='imagenet', include_top=False, input_tensor = input_tensor)

    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=(model.output_shape[1:])))


    # Output layer
    # We do random weight intialization
    top_model.add(Dropout(hp_dropout[setting]))
    top_model.add(Dense(hp_hidden[setting], activation='relu', kernel_initializer='glorot_uniform'))
    top_model.add(Dense(1, activation='linear', name='output', kernel_initializer='glorot_uniform'))

     # add the model on top of the convolutional base
    new_model = Model(inputs= model.input, outputs = top_model(model.output))

In [21]:
new_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            [(None, 299, 299, 3) 0                                            
__________________________________________________________________________________________________
conv2d_609 (Conv2D)             (None, 149, 149, 32) 864         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_609 (BatchN (None, 149, 149, 32) 96          conv2d_609[0][0]                 
__________________________________________________________________________________________________
activation_609 (Activation)     (None, 149, 149, 32) 0           batch_normalization_609[0][0]    
____________________________________________________________________________________________

In [22]:
new_model.layers

In [ ]:
for setting in range(num_settings):
    # set the first 19 layers (up to the last conv block)
    # to non-trainable (weights will not be updated)
    for layer in new_model.layers[:19]:
        layer.trainable = False

    # RMSprop optimizer
    new_model.compile(loss='mean_squared_error',
                      optimizer=optimizers.RMSprop(
                              lr=hp_lr[setting],
                              rho=hp_rho[setting],
                              epsilon=hp_epsilon[setting],
                              decay=hp_decay[setting]))

    checkpoint_path = '/output/ecommerce-cnn-best.hdf5'

    # keep a checkpoint
    checkpoint = ModelCheckpoint(checkpoint_path,
                                monitor='val_loss',
                                save_best_only=True,
                                mode='min')

    minibatch_size = hp_mbsize[setting]

    train_steps = math.ceil(len(train_indices) / minibatch_size)
    test_steps = math.ceil(len(test_indices) / minibatch_size)

    # fine-tune the model
    history = new_model.fit(
        image_generator(train_indices, minibatch_size),
        steps_per_epoch=train_steps,
        epochs=num_epochs,
        validation_data=image_generator(test_indices, minibatch_size),
        callbacks=[checkpoint])

    # store the training and dev losses for the last epoch (current model)
    train_losses[setting] = history.history['loss'][-1]
    dev_losses[setting] = history.history['val_loss'][-1]


    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()

    print("==========")